# Tuning the model


## Load the data
Load the preprocessed data which prepared in HDF5 format. The data is split into training, validation, and test sets. The training set is used to train the model, the validation set is used to tune the hyperparameters, and the test set is used to evaluate the final model performance.

In [1]:
from dataset import CHBMITPreprocessedDataset

dataset = CHBMITPreprocessedDataset('./CHB-MIT/processed_data.h5')

## Assemble the model

We will use the Pytorch Lightning library to build the model pipeline from the based model.

In [2]:
from dataset.lightning import CHBMITPreprocessedDataModule

datamodule = CHBMITPreprocessedDataModule(dataset, batch_size=32)

In [3]:
import optuna
from encoder.temporal import StepForwardEncoder
from optuna.integration import PyTorchLightningPruningCallback
from model.classifier import ModelConfig
from model.lightning import OptimizerConfig, LitSTFTPreprocessedSeizureClassifier
import lightning.pytorch as pl


def objective(trial: optuna.trial.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.5),
        "slope": trial.suggest_float("slope", 1.0, 20.0),
        "beta": trial.suggest_float("beta", 0.1, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.1, 0.99),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.99),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.99),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 1, 10),
    }
    encoder_threshold = trial.suggest_float("encoder_threshold", 0.1, 0.99, log=True)
    spike_encoder = StepForwardEncoder(threshold=encoder_threshold)

    model = LitSTFTPreprocessedSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=15,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss")],
    )

    trainer.fit(model, datamodule=datamodule)

    return trainer.callback_metrics["val_loss"].item()

In [4]:
from config import DB_CONN_STRING

sampler = optuna.samplers.CmaEsSampler()
study = optuna.create_study(
    direction="minimize",
    study_name="Classifier_SF_Model_tuning",
    sampler=sampler,
    storage=DB_CONN_STRING,
    load_if_exists=True,
    pruner=optuna.pruners.HyperbandPruner()
)

[I 2025-05-01 01:08:08,853] Using an existing study with name 'Classifier_SF_Model_tuning' instead of creating a new one.


In [5]:
study.optimize(objective, n_trials=50)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type               | Params | Mode 
-----------------------------------------------------
0 | model | EEGSpikeClassifier | 824 K  | train
-----------------------------------------------------
824 K     Trainable params
0         Non-trainable params
824 K     Total params
3.299     Total estimated model params size (MB)
13        

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...
[W 2025-05-01 01:37:25,558] Trial 12 failed with parameters: {'threshold': 0.2858945000166388, 'slope': 10.734244442742263, 'beta': 0.6563874592626484, 'dropout_rate1': 0.5410247848321447, 'dropout_rate2': 0.6325080663634957, 'lr': 7.613634684538611e-06, 'weight_decay': 7.921002354925385e-06, 'scheduler_factor': 0.5149688707459893, 'scheduler_patience': 7, 'encoder_threshold': 0.27992673867600865} because of the following error: NameError("name 'exit' is not defined").
Traceback (most recent call last):
  File "/home/jupyter-group55/.local/lib/python3.12/site-packages/lightning/pytorch/trainer/call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jupyter-group55/.local/lib/python3.12/site-packages/lightning/pytorch/trainer/trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/home/jupyter-group55/.local/lib

NameError: name 'exit' is not defined

In [ ]:
study.best_params, study.best_value